In [1]:
exec(open("../../../python/FNC_init.py").read())

[**Demo %s**](#demo-systems-triangular)


```{index} ! Python; tril, ! Python; triu
```

It's easy to get just the lower triangular part of any matrix using the `tril` function.

In [2]:
A = 1 + floor(9 * random.rand(5, 5))
L = tril(A)
print(L)

[[6. 0. 0. 0. 0.]
 [6. 9. 0. 0. 0.]
 [9. 4. 3. 0. 0.]
 [2. 2. 5. 7. 0.]
 [7. 3. 7. 6. 4.]]


We'll set up and solve a linear system with this matrix.

In [3]:
b = ones(5)
x = FNC.forwardsub(L, b)
print(x)

[ 0.16666667  0.         -0.16666667  0.21428571 -0.07142857]


It's not clear how accurate this answer is. However, the residual should be zero or comparable to $\macheps$.

In [4]:
b - L @ x

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.11022302e-16])

Next we'll engineer a problem to which we know the exact answer.

In [5]:
alpha = 0.3;
beta = 2.2;
U = diag(ones(5)) + diag([-1, -1, -1, -1], k=1)
U[0, 3:5] = [ alpha - beta, beta ]
print(U)

[[ 1.  -1.   0.  -1.9  2.2]
 [ 0.   1.  -1.   0.   0. ]
 [ 0.   0.   1.  -1.   0. ]
 [ 0.   0.   0.   1.  -1. ]
 [ 0.   0.   0.   0.   1. ]]


In [6]:
x_exact = ones(5)
b = array([alpha, 0, 0, 0, 1])
x = FNC.backsub(U, b)
print("error:", x - x_exact)

error: [2.22044605e-16 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00]


Everything seems OK here. But another example, with a different value for $\beta$, is more troubling.

In [7]:
alpha = 0.3;
beta = 1e12;
U = diag(ones(5)) + diag([-1, -1, -1, -1], k=1)
U[0, 3:5] = [ alpha - beta, beta ]
b = array([alpha, 0, 0, 0, 1])

x = FNC.backsub(U, b)
print("error:", x - x_exact)

error: [-4.8828125e-05  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00]


It's not so good to get 4 digits of accuracy after starting with sixteen! But the source of the error is not hard to track down. Solving for $x_1$ performs $(\alpha-\beta)+\beta$ in the first row. Since $|\alpha|$ is so much smaller than $|\beta|$, this a recipe for losing digits to subtractive cancellation.